In [1]:
# sys.stdout = open("FTL_MNIST_1.txt", "w")

In [2]:
# !pip3 install tensorflow_model_optimization

In [6]:
from tensorflow.keras.utils import to_categorical
import os
import sys
import tempfile

import cv2
import keras
import numpy as np
from numpy import dstack 
import pandas as pd
from keras.layers import (Conv2D, Dense, Dropout, Flatten, GaussianNoise,
                          MaxPooling2D, MaxPool2D , Activation)
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.models import Sequential
from keras.utils import np_utils
from matplotlib import pyplot
from numpy import dstack, mean, std
from pandas import read_csv
from PIL import Image
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
# from keras.utils import to_categorical
from sklearn.model_selection import train_test_split


%matplotlib inline
import random

import matplotlib.pyplot as plt
import numpy
import plotly.express as px
import requests
import tensorflow as tf
import tensorflow_model_optimization as tfmot
from scipy.spatial.distance import euclidean as euc

prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

from plotly.offline import download_plotlyjs, init_notebook_mode, iplot

init_notebook_mode(connected=True)

In [4]:
DF_train = pd.read_csv('mnist_train.csv')
DF_test = pd.read_csv('mnist_test.csv')

In [29]:
# ----------------------------- #
# ---------- SETTINGS ----------#
# ----------------------------- #

NUM_Clients = 5 # number of clients contributing per training round

# ML
Cluster_Size = 100 # max client dataset size for training
Batch_Size = 32
NUM_Epoch = 3
verbose = 1
TL_Epochs = 1

# Krum
krum_f = 0.00 # percentage of byzantine nodes

# Differential Privacy
Gaussian_Noise = False
Gaussian_Noise_Std_Dev = 0.20

Gradient_Clipping = False
Clip_Norm = 0.60

Gradient_Pruning = False
initial_sparsity = 0.00
final_sparsity = 0.50



# ---------------------------- #
# ----------------------------- #
# ----------------------------- #
def settings():
    global NUM_Clients,Cluster_Size,Batch_Size,NUM_Epoch,verbose,krum_f,Gaussian_Noise,Gaussian_Noise_Std_Dev,Gradient_Clipping,Clip_Norm,Gradient_Pruning,initial_sparsity,final_sparsity

In [30]:
def preprocess(df):
    y = df.label
    X = df.drop(['label'], axis=1)
    X = X.values.reshape(-1,28 ,28,1)
    X = X/255.0
    y = to_categorical(y, num_classes=10)
    
    print('Data size : ', X.shape)
    return X, y

DF_Train_X,DF_Train_Y = preprocess(DF_train)
DF_Test_X,DF_Test_Y = preprocess(DF_test) 

def split_70_30(X,y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)
    return  X_train, X_test, y_train, y_test 

Data size :  (60000, 28, 28, 1)
Data size :  (10000, 28, 28, 1)


In [31]:
def train(name, X_train, y_train, globalId):
    global curr_global
    global curr_local

    n_timesteps, n_features, n_outputs = X_train.shape[0], X_train.shape[1], y_train.shape[0]

    model = Sequential()

    model.add(Conv2D(filters=16,kernel_size=2,padding="same",activation="relu",input_shape=(28,28,1)))

    if Gaussian_Noise == True:
        model.add(GaussianNoise(Gaussian_Noise_Std_Dev))

    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=32,kernel_size=2,padding="same",activation="relu"))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=64,kernel_size=2,padding="same",activation="relu"))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(500,activation="relu"))
    model.add(Dropout(0.2))
    model.add(Dense(10,activation="softmax"))#2 represent output layer neurons
    if Gradient_Pruning == True:
        end_step = np.ceil(n_timesteps / Batch_Size).astype(np.int32) * NUM_Epoch

        # Define model for pruning.
        pruning_params = {
              'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=initial_sparsity,
                                                                       final_sparsity=final_sparsity,
                                                                       begin_step=0,
                                                                       end_step=end_step)
        }

        logdir = tempfile.mkdtemp()

        callbacks = [
          tfmot.sparsity.keras.UpdatePruningStep(),
          tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
        ]

        model = prune_low_magnitude(model, **pruning_params)

    model.built = True

    if globalId != 1:
        model.load_weights("./weights/global"+str(globalId)+".h5")

    if Gradient_Clipping == True:
        opt = keras.optimizers.Adam(clipnorm=Clip_Norm)
        model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    else: 
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    if Gradient_Pruning == True:
        history = model.fit(X_train, y_train, batch_size=Batch_Size, epochs=NUM_Epoch, verbose=1, callbacks=callbacks)
    else:
        history = model.fit(X_train, y_train, batch_size=Batch_Size, epochs=NUM_Epoch, verbose=1)

    #Saving Model
    model.save("./weights/"+str(name)+".h5")
    return n_timesteps, model

In [32]:
def euclidean(m, n):
    global curr_global
    global curr_local
    # Finds eucledian distance between two ML models m & n
    distance = []
    for i in range(len(m)):
        distance.append(euc(m[i].reshape(-1,1), n[i].reshape(-1,1)))
    distance = sum(distance)/len(m)
    return distance

def saveModel(weight, n):
    global curr_global
    global curr_local
    
    num_classes=len(np.unique(DF_Test_Y))

    model = Sequential()
    model.add(Conv2D(filters=16,kernel_size=2,padding="same",activation="relu",input_shape=(28,28,1)))
    if Gaussian_Noise == True: model.add(GaussianNoise(Gaussian_Noise_Std_Dev))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=32,kernel_size=2,padding="same",activation="relu"))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=64,kernel_size=2,padding="same",activation="relu"))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(500,activation="relu"))
    model.add(Dropout(0.2))
    model.add(Dense(10,activation="softmax"))#2 represent output layer neurons

    if Gradient_Pruning == True:
        end_step = np.ceil(Cluster_Size/Batch_Size).astype(np.int32) * NUM_Epoch

          # Define model for pruning.
        pruning_params = {
            'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=initial_sparsity,
                                                                        final_sparsity=final_sparsity,
                                                                        begin_step=0,
                                                                        end_step=end_step)
          }
        logdir = tempfile.mkdtemp()

        callbacks = [
            tfmot.sparsity.keras.UpdatePruningStep(),
            tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
        ]

        model = prune_low_magnitude(model, **pruning_params)

    model.set_weights(weight)

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    scores = model.evaluate(DF_Test_X, DF_Test_Y)

    print("Saved Model Loss: ", scores[0])        #Loss
    print("Saved Model Accuracy: ", scores[1])    #Accuracy

    #Saving Model
    fpath = "./weights/global"+str(n)+".h5"
    model.save(fpath)
    return scores[0], scores[1]

def getDataLen(trainingDict):
    global curr_global
    global curr_local
    n = 0
    for w in trainingDict:
        n += trainingDict[w]
#     print('Total number of data points after this round: ', n)
    return n

def assignWeights(trainingDf, trainingDict):
    global curr_global
    global curr_local
    n = getDataLen(trainingDict)
    trainingDf['Weightage'] = trainingDf['DataSize'].apply(lambda x: x/n)
    return trainingDf, n
    
def scale(weight, scaler):
    global curr_global
    global curr_local
    scaledWeights = []
    for i in range(len(weight)):
        scaledWeights.append(scaler * weight[i])
    return scaledWeights

def getScaledWeight(d, scaler):
    global curr_global
    global curr_local
    model = Sequential()

    model.add(Conv2D(filters=16,kernel_size=2,padding="same",activation="relu",input_shape=(28,28,1)))

    if Gaussian_Noise == True:
        model.add(GaussianNoise(Gaussian_Noise_Std_Dev))

    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=32,kernel_size=2,padding="same",activation="relu"))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=64,kernel_size=2,padding="same",activation="relu"))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(500,activation="relu"))
    model.add(Dropout(0.2))
    model.add(Dense(10,activation="softmax"))#2 represent output layer neurons

    if Gradient_Pruning == True:
        model = prune_low_magnitude(model)
    fpath = "./weights/"+d+".h5"
    model.load_weights(fpath)
    weight = model.get_weights()
    return scale(weight, scaler)

def getWeight(d):
    global curr_global
    global curr_local
    model = Sequential()

    model.add(Conv2D(filters=16,kernel_size=2,padding="same",activation="relu",input_shape=(28,28,1)))

    if Gaussian_Noise == True:
        model.add(GaussianNoise(Gaussian_Noise_Std_Dev))

    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=32,kernel_size=2,padding="same",activation="relu"))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=64,kernel_size=2,padding="same",activation="relu"))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(500,activation="relu"))
    model.add(Dropout(0.2))
    model.add(Dense(10,activation="softmax"))#2 represent output layer neurons

    if Gradient_Pruning == True:
        model = prune_low_magnitude(model)
    fpath = "./weights/"+d+".h5"
    model.load_weights(fpath)
    weight = model.get_weights()
    return weight

def avgWeights(scaledWeights):
    global curr_global
    global curr_local
    avg = list()
    for weight_list_tuple in zip(*scaledWeights):
        layer_mean = tf.math.reduce_sum(weight_list_tuple, axis=0)
        avg.append(layer_mean)
    return avg

def FedAvg(trainingDict):
    global curr_global
    global curr_local
    trainingDf = pd.DataFrame.from_dict(trainingDict, orient='index', columns=['DataSize']) 
    models = list(trainingDict.keys())
    scaledWeights = []
    trainingDf, dataLen = assignWeights(trainingDf, trainingDict)
    for m in models:
        scaledWeights.append(getScaledWeight(m, trainingDf.loc[m]['Weightage']))
    fedAvgWeight = avgWeights(scaledWeights)
    return fedAvgWeight, dataLen

def MK(trainingDict, b):
    global curr_global
    global curr_local
    print('MK Training Dict: ', trainingDict)
    models = list(trainingDict.keys())
    trainingDf = pd.DataFrame.from_dict(trainingDict, orient='index', columns=['DataSize'])
    l_weights = []
    g_weight = {}
    for m in models:
        if 'global' in m:
            g_weight['name'] = m
            g_weight['weight'] = getWeight(m)
        else:
            l_weights.append({
                'name': m,
                'weight': getWeight(m)
            })
    
    scores = {}
#     if (g_weight == {}):
#         return -1,-1
    for m in l_weights:
        scores[m['name']] = euclidean(m['weight'], g_weight['weight'])
    sortedScores = {k: v for k, v in sorted(scores.items(), key=lambda item: item[1])}

    b = int(len(scores)*b)
    
    selected = []
    for i in range(b):
        selected.append((sortedScores.popitem())[0])

    newDict = {}
    for i in trainingDict.keys():
        if (((i not in selected) and ('global' not in i))):
            newDict[i] = trainingDict[i]

    print('Selections: ', newDict)
    NewGlobal, dataLen = FedAvg(newDict)
    return NewGlobal, dataLen

def TransferLearn(name, X_train, X_test,y_train,y_test):
    global curr_global
    global curr_local
    # X_train, y_train = preprocess(traindf)
    # X_test, y_test = preprocess(testdf)

    n_timesteps, n_features, n_outputs = X_train.shape[0], X_train.shape[1], y_train.shape[0]

    inner_model = Sequential(
    [
        Conv2D(filters=16,kernel_size=2,padding="same",activation="relu",input_shape=(28,28,1))
    ]
    )

    model = Sequential()
    model.add(inner_model)

    if Gaussian_Noise == True:
        model.add(GaussianNoise(Gaussian_Noise_Std_Dev))

    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=32,kernel_size=2,padding="same",activation="relu"))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=64,kernel_size=2,padding="same",activation="relu"))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(500,activation="relu"))
    model.add(Dropout(0.2))
    model.add(Dense(10,activation="softmax"))#2 represent output layer neurons


    if Gradient_Pruning == True:
        end_step = np.ceil(n_timesteps / Batch_Size).astype(np.int32) * NUM_Epoch

        # Define model for pruning.
        pruning_params = {
              'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=initial_sparsity,
                                                                       final_sparsity=final_sparsity,
                                                                       begin_step=0,
                                                                       end_step=end_step)
        }

        logdir = tempfile.mkdtemp()

        callbacks = [
          tfmot.sparsity.keras.UpdatePruningStep(),
          tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
        ]

        model = prune_low_magnitude(model, **pruning_params)

    model.load_weights("./weights/global"+str(curr_global)+".h5")

    if Gradient_Clipping == True:
        opt = keras.optimizers.Adam(clipnorm=Clip_Norm)
        model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    else: 
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    for layer in inner_model.layers:#freezing layers to retain the weights 
        layer.trainable = False

    if Gradient_Pruning == True:
        history = model.fit(X_train, y_train, epochs=TL_Epochs, batch_size=Batch_Size, verbose=1, callbacks=callbacks)
    else:
        history = model.fit(X_train, y_train, epochs=TL_Epochs, batch_size=Batch_Size, verbose=1)

    history2 = model.evaluate(X_test, y_test, batch_size=Batch_Size, verbose=1)

    test_accuracy = history2[1]
    print(f"Subject Test Accuracy Post Transfer Learning is {test_accuracy}")

    nofedAcc = get_subject_testacc_before_TL(X_test, y_test, curr_global)

    getCent = get_own_individual_acc(X_train, y_train, curr_global)

    #Saving Model
    model.save("./weights/"+str(name)+".h5")
    return n_timesteps, model

def get_subject_testacc_before_TL(X_test, y_test, curr_global):
    
    global curr_local
#     global curr_global
        
    num_classes=len(np.unique(y_test))

    model = Sequential()

    model.add(Conv2D(filters=16,kernel_size=2,padding="same",activation="relu",input_shape=(28,28,1)))

    if Gaussian_Noise == True:
        model.add(GaussianNoise(Gaussian_Noise_Std_Dev))

    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=32,kernel_size=2,padding="same",activation="relu"))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=64,kernel_size=2,padding="same",activation="relu"))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(500,activation="relu"))
    model.add(Dropout(0.2))
    model.add(Dense(10,activation="softmax"))#2 represent output layer neurons

    if Gradient_Pruning == True:
        end_step = np.ceil(Cluster_Size / Batch_Size).astype(np.int32) * NUM_Epoch

        # Define model for pruning.
        pruning_params = {
            'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=initial_sparsity,
                                                                    final_sparsity=final_sparsity,
                                                                    begin_step=0,
                                                                    end_step=end_step)
        }

        logdir = tempfile.mkdtemp()

        callbacks = [
        tfmot.sparsity.keras.UpdatePruningStep(),
        tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
        ]

        model = prune_low_magnitude(model, **pruning_params)

    model.load_weights("./weights/global"+str(curr_global)+".h5")

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
      
    scores = model.evaluate(X_test,y_test)
    print("Subject Testing Loss before TL: ", scores[0])        #Loss
    print("Subject Testing Accuracy before TL: ", scores[1])    #Accuracy
    
def get_own_individual_acc(X_train, y_train, curr_global):
    
    global curr_local
#     global curr_global

    X_test, y_test = DF_Test_X,DF_Test_Y

    num_classes=len(np.unique(y_test))

    model = Sequential()

    model.add(Conv2D(filters=16,kernel_size=2,padding="same",activation="relu",input_shape=(28,28,1)))

    if Gaussian_Noise == True:
        model.add(GaussianNoise(Gaussian_Noise_Std_Dev))

    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=32,kernel_size=2,padding="same",activation="relu"))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=64,kernel_size=2,padding="same",activation="relu"))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(500,activation="relu"))
    model.add(Dropout(0.2))
    model.add(Dense(10,activation="softmax"))#2 represent output layer neurons

    if Gradient_Pruning == True:
        end_step = np.ceil(16500 / Batch_Size).astype(np.int32) * NUM_Epoch

        # Define model for pruning.
        pruning_params = {
            'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=initial_sparsity,
                                                                    final_sparsity=final_sparsity,
                                                                    begin_step=0,
                                                                    end_step=end_step)
        }

        logdir = tempfile.mkdtemp()

        callbacks = [
            tfmot.sparsity.keras.UpdatePruningStep(),
            tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
        ]

        model = prune_low_magnitude(model, **pruning_params)

    # model.load_weights("./weights/global"+str(curr_global)+".h5")

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    if Gradient_Pruning == True:
        history = model.fit(X_train, y_train, batch_size=Batch_Size, epochs=NUM_Epoch, verbose=1, callbacks=callbacks)
    else:
        history = model.fit(X_train, y_train, batch_size=Batch_Size, epochs=NUM_Epoch, verbose=1)
      
    scores = model.evaluate(X_test, y_test)
    print("Invidual Loss without TL: ", scores[0])        #Loss
    print("invidual Accuracy without TL: ", scores[1])    #Accuracy

In [33]:
def init():
    global NUM_Clients,Cluster_Size,Batch_Size,NUM_Epoch,verbose,krum_f,Gaussian_Noise,Gaussian_Noise_Std_Dev,Gradient_Clipping,Clip_Norm,Gradient_Pruning,initial_sparsity,final_sparsity
    
    num_iter = 2
    
    for i in range(num_iter):
        print(f"-----------------------Repeating entire process the {i}th time-------------------")
        global curr_local
        global curr_global
        
        print('----------------------------------------')
        print('Number of Clients: ', NUM_Clients)
        print('Cluster Size: ', Cluster_Size)
        print('Batch Size: ', Batch_Size)
        print('Number of Local Epochs: ', NUM_Epoch)
        print('F: ', krum_f)
        print('Gaussian_Noise: ', Gaussian_Noise)
        if Gaussian_Noise: 
            print('Noise Std Dev: ', Gaussian_Noise_Std_Dev)
        print('Gradient_Clipping: ', Gradient_Clipping)
        if Gradient_Clipping: 
            print('Clip Norm: ', Clip_Norm)
        print('Gradient_Pruning: ', Gradient_Pruning)
        if Gradient_Pruning: 
            print('Pruning Sparcity: ', final_sparsity)
        print('----------------------------------------')
        
        curr_local = 0
        curr_global = 0
        
        clients = {}
        
        Outer_Xtrain,Outer_Xtest,Outer_Ytrain,Outer_Ytest = DF_Train_X,DF_Test_X,DF_Train_Y,DF_Test_Y #happens at global level, only 70% is shared w clients, rest is kept for testing

        per_client_data = (len(Outer_Xtrain)//NUM_Clients)
        for i in range(NUM_Clients):
            
            clients[f'X_{i}'] = Outer_Xtrain[per_client_data*i: per_client_data*(i+1)]
            clients[f'Y_{i}'] = Outer_Ytrain[per_client_data*i: per_client_data*(i+1)]
            clients[f'X_train_{i}'],clients[f'X_test_{i}'],clients[f'Y_train_{i}'],clients[f'Y_test_{i}'] = split_70_30(clients[f'X_{i}'],clients[f'Y_{i}']) 

        local = {}
        loss_array = []
        acc_array = []
        curr_datalen = 0
        
        for i in range(0, len(Outer_Xtrain), Cluster_Size):
            curr_datalen += Cluster_Size
            print("Total Data Used: ", curr_datalen)
            
            if int(curr_global) == 0:
                curr_global += 1
                print('Current Global: ', curr_global)
                name = 'global' + str(curr_global)
                X_train_fed, y_train_fed = Outer_Xtrain[i:i+Cluster_Size],Outer_Ytrain[i:i+Cluster_Size]
                l, m = train(name, X_train_fed, y_train_fed, curr_global)
                local[name] = l
                
            else:
                print('Current Local: ', curr_local)
                name = str('local'+str(curr_local))
                curr_local += 1
                X_train_fed, y_train_fed = Outer_Xtrain[i:i+Cluster_Size],Outer_Ytrain[i:i+Cluster_Size]
                if X_train_fed.shape[0]<=Cluster_Size-1:
                    break
                l, m = train(name, X_train_fed, y_train_fed, curr_global)
                local[name] = l
                
                if (int(curr_local)% NUM_Clients == 0):
                    curr_global += 1
                    print('Current Global: ', curr_global)
                    name = 'global' + str(curr_global)
                    m, l = MK(local, krum_f)
                    loss, acc = saveModel(m, curr_global)
                    loss_array.append(loss)
                    acc_array.append(acc)
                    local = {}
                    local[name] = l

        print("Global Accuracy Array: ", acc_array)
          
                
        for j in range(NUM_Clients):
            print("calling tflearn for client ", j)
            TransferLearn(f"C{j}", clients[f'X_train_{j}'], clients[f'X_test_{j}'],clients[f'Y_train_{j}'],clients[f'Y_test_{j}'])

        NUM_Clients+=5

In [34]:
init()

-----------------------Repeating entire process the 0th time-------------------
----------------------------------------
Number of Clients:  10
Cluster Size:  1000
Batch Size:  32
Number of Local Epochs:  1
F:  0.0
Gaussian_Noise:  False
Gradient_Clipping:  False
Gradient_Pruning:  False
----------------------------------------
Total Data Used:  1000
Current Global:  1
32/32 [==============================] - 1s 20ms/step - loss: 2.2062 - accuracy: 0.1905
Total Data Used:  2000
Current Local:  0
32/32 [==============================] - 1s 22ms/step - loss: 2.2177 - accuracy: 0.1797
Total Data Used:  3000
Current Local:  1
32/32 [==============================] - 1s 25ms/step - loss: 2.2089 - accuracy: 0.2607
Total Data Used:  4000
Current Local:  2
32/32 [==============================] - 2s 28ms/step - loss: 2.2001 - accuracy: 0.2180
Total Data Used:  5000
Current Local:  3
32/32 [==============================] - 2s 34ms/step - loss: 2.1690 - accuracy: 0.2477
Total Data Used:  6000
C